In [34]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# CRUD Python module
from CRUD_Functions import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Loggin credentials for MongoDB
username = "aacuser"
password = "NotAPassWord"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Layout of the web page
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(
        style={
            'display': 'flex', 
            'justifyContent': 'space-between', 
            'alignItems': 'flex-end',
            'padding': '10px'
        },
        children=[
            # Logo
            html.Div(
                children=html.A(
                    html.Img(
                        src='data:image/png;base64,{}'.format(encoded_image.decode()),
                        style={'width': '15%', 'height': 'auto'}
                    ),
                    href='https://www.snhu.edu',
                    target="_blank"
                ),
                style={'flex': '1', 'textAlign': 'left'}
            ),
            # Dashboard title
            html.Div(
                children=html.H1('SNHU CS-340 Dashboard', style={'margin': 0}),
                style={'flex': '1', 'textAlign': 'center'}
            ),
            # Unique Identifier
            html.Div(
                children=html.H2([
                    "Created by: ", 
                    html.Span("Austin Sarkis", style={'color': 'red'})
                ], style={'margin': 0}),
                style={'flex': '1', 'textAlign': 'right'}
            )
        ]
    ),
    html.Hr(),
    # Filter Options
    html.Div(
        id='filter-container',
        style={'display': 'flex', 'flexDirection': 'row', 'justifyContent': 'center', 'alignItems': 'center'},
        children=[
            html.Span("Mission options: ", style={'marginRight': '10px'}),
            dcc.RadioItems(
                id='mission-filter',
                options=[
                    {'label': 'Water Rescue', 'value': 'Water Rescue'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                    {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue or Individual Tracking'},
                    {'label': 'Reset', 'value': 'Reset'}
                ],
                value='Reset',
                labelStyle={'display': 'inline-block', 'margin-right': '20px'}
            )
        ]
    ),
    
    # Data from database
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),
    
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            style={'width': '50%'}

            ),
        html.Div(
            id='map-id',
            style={'width': '50%'}
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################   
# Callback to filter the data table when one of the radio buttons are selected.
@app.callback(
    Output('datatable-id', "data"),
    [Input('mission-filter', 'value')]
)
def update_table(selected_mission):
    if selected_mission is None or selected_mission == "Reset":
        query = {}
    else:
        if selected_mission == "Water Rescue":
            query = {
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix",
                                  "Chesapeake Bay Retriever",
                                  "Newfoundland"]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
            }
        elif selected_mission == "Mountain or Wilderness Rescue":
            query = {
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepherd",
                                  "Alaskan Malamute",
                                  "Old English Sheepdog",
                                  "Siberian Husky",
                                  "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
            }
        elif selected_mission == "Disaster Rescue or Individual Tracking":
            query = {
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher",
                                  "German Shepherd",
                                  "Golden Retriever",
                                  "Bloodhound",
                                  "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
            }

    df_filtered = pd.DataFrame.from_records(shelter.read(query))
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)
    return df_filtered.to_dict('records')

# Callback to update the pie graph
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return []
    
    # Convert the data to a DataFrame
    df_local = pd.DataFrame.from_records(viewData)
    
    # Group by 'breed', count occurrences, and select the top 10 breeds
    breed_counts = df_local['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']
    top_breeds = breed_counts.head(10)
    
    # Use Plotly's built-in Prism qualitative color palette
    custom_colors = px.colors.qualitative.Prism
    
    # Create a pie chart using the top 10 breeds and the Prism color palette
    fig = px.pie(top_breeds,
                 names='breed',
                 values='count',
                 color_discrete_sequence=custom_colors)
    
    # Add a title with a subtitle
    fig.update_layout(
        title={
            'text': "Top 10 Breeds<br><sub>Distribution breakdown in current selection</sub>",
            'x': 0.5
        }
    )
    
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None or index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Use the marker's latitude and longitude from the selected record as the new center
    new_center = [dff.iloc[row, 13], dff.iloc[row, 14]]
        
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=new_center, zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:12488/
